In [ ]:
import logging
LOG_FILENAME = '/core/logs/backtest.log'
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',filename=LOG_FILENAME)
from cdcqr.backtest.vbt.runsignal import runsignal
from cdcqr.common.utils import parallel_jobs
import numpy as np
from cdcqr.common.utils import LOCAL_DATA_DIR, dict_hash
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from cdcqr.common.utils import LOCAL_DATA_DIR
import json
from cdcqr.data.glassnode.features.feature_loaders import produce_feature_BTCSSR
from cdcqr.backtest.vbt.sample_params import param_dict

pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
from cdcqr.backtest.vbt.runsignal import wrapped_runsignal


In [ ]:
param_dict_raw = param_dict.copy()
param_dict_raw

In [ ]:
feature_data_fname = 'gn_BTCSSR'
feature_data_dir = os.path.join(LOCAL_DATA_DIR, 'feature_data', '{}.pickle'.format(feature_data_fname))
df4a = pd.read_pickle(feature_data_dir)

In [ ]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))


In [ ]:
qtl_params = [0.995, 0.99, 0.975, 0.96]
lbw_params = [1000, 2000, 3000, 4000]

tp_params = [3, 5, 7, 10] # 
sl_params = [1, 2, 3 , 5] # ts = sl 
n1_params = [1, 10, 100]
n2_params = [101, 200, 500, 1000, 2000]
rsil_params = [10, 20]
rsilag_params = [14]
maf_params = [5, 60]
mas_params = [100, 200]

In [ ]:
qtl_params = [0.995, 0.99]
lbw_params = [1000, 2000]

tp_params = [3, 5, ] # 
sl_params = [1, 2,] # ts = sl 
n1_params = [1, 10,]
n2_params = [101, 200]

rsil_params = [10, 20]
rsilag_params = [14]
maf_params = [5, 60]
mas_params = [100, 200]


In [ ]:
feature = 'BTCSSR'

In [ ]:
counter = 0
backtest_configs = []
for lbw in lbw_params:
    feature_pctrank = '{}_pctrank{}'.format(feature, lbw)
    df4a[feature_pctrank] = df4a[feature].rolling(lbw).apply(pctrank)
    df4b = df4a[df4a.index>='20210101']
    for qtl in qtl_params:
        signal = df4b[feature_pctrank]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
        signal = signal.reindex(price.index).ffill()
        for tp in tp_params:
            for sl in sl_params:
                print(qtl, lbw, tp, sl)
                for n1 in n1_params:
                    for n2 in n2_params:
                        counter+=1
                        print(counter)
                        for rsil in rsil_params:
                            rsih = 100-rsil
                            for maf in maf_params:
                                for mas in mas_params:
                                    for railag in rsilag_params:
                                        
                                        param_dict = param_dict_raw.copy()
                                        param_dict['qtl'] = qtl
                                        param_dict['lbw'] = lbw
                                        param_dict['signal'] = signal
                                        param_dict['price'] = price
                                        param_dict['tp'] = tp
                                        param_dict['sl'] = sl
                                        param_dict['n1'] = n1
                                        param_dict['n2'] = n1 + n2
                                        param_dict['rsil'] = rsil
                                        param_dict['rsih'] = rsih
                                        param_dict['maf'] = maf
                                        param_dict['mas'] = mas
                                        param_dict['railag'] = railag
                                        
                                        backtest_configs.append(param_dict)

In [ ]:
len(backtest_configs)

In [ ]:
bt_config = backtest_configs[0]

In [ ]:
ret = wrapped_runsignal(bt_config)

In [1]:
# 16
qtl_params = [0.995, 0.99, 0.975, 0.96]
lbw_params = [1000, 2000, 3000, 4000]

# 16
tp_params = [3, 5, 7, 10] # 
sl_params = [1, 2, 3 , 5] # ts = sl 

# 15
n1_params = [1, 10, 100]
n2_params = [100, 200, 500, 1000, 2000]

# 2
rsil_params = [10, 20] # 2 runs with rsil rsilag
rsilag_params = [14]

# 4
maf_params = [5, 60] # 4 runs for maf/mas
mas_params = [100, 200]

print(len(qtl_params)*len(lbw_params)*len(tp_params)*len(sl_params)*len(n1_params)*len(n2_params)*len(rsil_params)*len(rsilag_params)*len(maf_params)*len(mas_params))

30720


In [ ]:
8->6

